## BeautifulSoup 으로 웹페이지 가져오기

requests 패키지는 url 주소의 정보를 가져옵니다. 여기에는 headers 나 cookies 와 같은 정보도 포함되어 있습니다. 우리가 웹 브라우저로 보는 HTML 코드는 text 에 저장되어 있습니다.

HTML 은 XML 로 구조화된 텍스트입니다. 그렇기 때문에 XML parser 를 이용하면 문서의 각 부분을 손쉽게 접근할 수 있습니다. BeautifulSoup 은 이를 도와주는 패키지 입니다. 현재 최신 버전은 4.8.1 이며, 4.6.x 에서 4.7.x 로 넘어오면서 바뀐 문법이 있습니다. 반드시 버전을 확인하시기 바랍니다. 이전 버전일 경우에 pip 을 이용하여 업그레이드 할 수 있습니다.

```
pip install --upgrade beautifulsoup4
pip install beautifulsoup4==4.7.1
```

In [1]:
import bs4
from bs4 import BeautifulSoup

print(bs4.__version__)

4.8.1


In [2]:
import requests

# 영화 라라랜드 (배우, 제작진)
url = 'https://movie.naver.com/movie/bi/mi/detail.nhn?code=134963'
r = requests.get(url)

In [3]:
print(r.headers, end='\n\n')
print(r.cookies)

{'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Cache-Control': 'no-cache, no-store', 'Content-Language': 'ko-KR', 'P3P': 'CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC", CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=UTF-8', 'Content-Length': '30217', 'Accept-Ranges': 'bytes', 'Date': 'Fri, 17 Jan 2020 20:36:57 GMT', 'X-Varnish': '1147081373', 'Age': '0', 'Via': '1.1 varnish', 'X-Varnish-Cache': 'MISS', 'Referrer-Policy': 'unsafe-url', 'Server': 'nfront'}

<RequestsCookieJar[]>


requests 를 통하여 url 에 접속한 뒤 HTML 파일을 선택합니다. BeautifulSoup 을 이용하여 이를 구조화 하는 과정까지 이후로 반복할 것이기 때문에 `get_soup` 이라는 함수를 만듭니다.

In [4]:
def get_soup(url, headers=None):
    try:
        r = requests.get(url, headers=headers).text
        return BeautifulSoup(r, 'lxml')
    except Exception as e:
        print(e)
        return None

soup = get_soup(url)

## 영화 제목 선택하기

![](lalaland_main.png)

좌측 상단에 영화 이름이 있습니다. 원하는 정보를 드래그한 뒤, 크롬의 Inspect (한글은 요소 탐색) 을 눌러보시면 해당 부분의 source code 가 우측에 하이라이팅 되어 나타납니다. La La Land 라는 영화 제목은 strong 이라는 태그 안에 들어있으며, 그 태그의 class 는 `h_movie2` 입니다. HTML 에서 태그는 `<` 으로 시작하여 `>` 으로 끝나는 부분입니다. 링크의 경우에는 `<a>` 로 시작하여 `</a>` 로 끝납니다. 

`<strong class=h_movie2">` 는 `<div class=mv_info>` 아래에 있다는 것도 볼 수 있습니다. 

```python
soup.select('div[class=mv_info] strong[class=h_movie2]')
```

위 코드는 `mv_info` 라는 클래스 이름을 갖는 `div` 아래에 속한, class 이름이 `h_movie2` 인 strong tag 를 선택한다는 의미입니다. 해당 조건을 만족하는 결과가 하나 이상일 수 있기에 list 형식으로 return 됩니다. 위 조건에 만족하는 부분이 두 개가 있습니다. 이들을 for loop 으로 돌면서 텍스트만 선택합니다.

In [5]:
title_items = soup.select('div[class=mv_info] strong[class=h_movie2]')

for item in title_items:
    print(item.text)

La La Land
					
					, 
					2016
La La Land, 2016


텍스트 안에는 줄바꿈이나 tap 과 같은 기호들이 많습니다. 이들을 제거하는 normalize 함수를 만듭니다. Regular expression 을 이용할 수 있습니다. 혹은 replace 함수를 이용하여도 좋습니다. 두 개의 item 에 같은 정보가 포함되어 있으니, 우리는 하나만 이용합니다.

In [6]:
import re

normalize_pattern = re.compile('[\n\t]')
doublespace_pattern = re.compile('\s+')

def normalize(text):
    text = normalize_pattern.sub(' ', text)
    text = doublespace_pattern.sub(' ', text)
    return text.strip()

for item in title_items:
    text = normalize(item.text)
    print(text)

La La Land , 2016
La La Land, 2016


이 부분까지를 정리하여 soup 에서 영화 제목을 파싱하는 함수를 만듭니다.

In [7]:
def parse_title(soup):
    title = soup.select('div[class=mv_info] strong[class=h_movie2]')[-1].text
    title = normalize(title)
    return title

parse_title(soup)

'La La Land, 2016'

그 외의 개요, 감독, 등급 등의 정보도 위와 같이 각 정보가 위치한 HTML tag 를 탐색한 뒤, 함수를 만들면 됩니다.

## 배우 리스트 선택하기

배우를 살펴보면 `class=lst_people` 아래에 각각 `li` 로 나뉘어져 있음을 확인할 수 있습니다. 그런데 li 아래에 또 li 가 있기 때문에 각 배우에 대한 정보가 나뉘어질 수 있습니다. 대신에 `class=p_info` 라는 `div` 가 각 배우의 정보를 모두 감싸고 있으니 이를 이용할 수 있습니다.

In [8]:
actor_items = soup.select('ul[class=lst_people] div[class=p_info]')
# actor_items = soup.select('ul[class=lst_people] li')
len(actor_items)

12

`href^=AAA` 는 AAA 로 href 의 값이 시작하는 경우를 의미합니다. 이처럼 BeautifulSoup 은 다양한 문법을 제공합니다. 이들을 확인하려면 [cookbook](https://beautiful-soup-4.readthedocs.io/en/latest/) 을 보시기 바랍니다.

**참고**

BeautifulSoup <= 4.6.1 에서는 href 에 아래처럼 값을 입력하여도 파싱이 되었습니다.

```python
actor_items[0].select('a[href^=/movie/bi/pi/basic.nhn?code=]')
```

하지만 4.7.1 부터는 반드시 `"` 을 이용하여 href 의 값을 감싸야 작동합니다.

In [9]:
actor_items[0].select('a[href^="/movie/bi/pi/basic.nhn?code="]')

[<a class="k_name" href="/movie/bi/pi/basic.nhn?code=5751" title="라이언 고슬링">라이언 고슬링</a>]

위에서 연습한 것처럼 배우의 이름, 역할, 주연/조연 유무를 파싱하는 함수를 만듭니다. soup.select 의 결과도 list of soup 이기 때문에 select 함수를 이용할 수 있습니다. 어떤 영화에서는 주연, 조연 정보가 포함되어 있지 않을 수도 있기 때문에 try - except 구문을 이용합니다. select 함수에 매칭되는 부분이 없는 경우에는 empty str 를 name 에 입력합니다.

Return type 은 이후에 정보를 추가하기 좋도록 JSON 형식을 이용합니다.

In [10]:
def parse_actor(item):
    try:
        name = normalize(item.select('a[href^="/movie/bi/pi/basic.nhn?code="]')[0].text)
    except:
        name = ''
    
    try:
        role = normalize(item.select('p[class=pe_cmt]')[0].text)
    except:
        role = ''

    return {'name': name, 'role': role}

for item in actor_items:
    print(parse_actor(item))

{'name': '라이언 고슬링', 'role': '세바스찬 역'}
{'name': '엠마 스톤', 'role': '미아 역'}
{'name': '존 레전드', 'role': '키이스 역'}
{'name': '로즈마리 드윗', 'role': '로라 역'}
{'name': 'J.K. 시몬스', 'role': '빌- 재즈클럽 사장 역'}
{'name': '소노야 미즈노', 'role': '케이틀린 역'}
{'name': '제시카 로테', 'role': '알렉시스 역'}
{'name': '칼리 헤르난데스', 'role': '트레이시 역'}
{'name': '톰 에버렛 스콧', 'role': '데이비드 역'}
{'name': '핀 위트록', 'role': '그렉 역'}
{'name': '제이슨 푸치스', 'role': '카를로 역'}
{'name': '올리비아 해밀턴', 'role': '브리 역'}


여기에 order 를 추가해도 좋을 것 같습니다. order 가 작을수록 영화에서 비중이 높다는 의미입니다. 이를 위하여 enumerate 를 이용합니다.

In [11]:
for i, item in enumerate(actor_items):
    order = i + 1
    actor_info = parse_actor(item)
    actor_info['order'] = order
    print(actor_info)

{'name': '라이언 고슬링', 'role': '세바스찬 역', 'order': 1}
{'name': '엠마 스톤', 'role': '미아 역', 'order': 2}
{'name': '존 레전드', 'role': '키이스 역', 'order': 3}
{'name': '로즈마리 드윗', 'role': '로라 역', 'order': 4}
{'name': 'J.K. 시몬스', 'role': '빌- 재즈클럽 사장 역', 'order': 5}
{'name': '소노야 미즈노', 'role': '케이틀린 역', 'order': 6}
{'name': '제시카 로테', 'role': '알렉시스 역', 'order': 7}
{'name': '칼리 헤르난데스', 'role': '트레이시 역', 'order': 8}
{'name': '톰 에버렛 스콧', 'role': '데이비드 역', 'order': 9}
{'name': '핀 위트록', 'role': '그렉 역', 'order': 10}
{'name': '제이슨 푸치스', 'role': '카를로 역', 'order': 11}
{'name': '올리비아 해밀턴', 'role': '브리 역', 'order': 12}


이 과정을 정리하여 출연 배우 리스트를 파싱하는 parse_actors 함수를 만듭니다.

In [12]:
def parse_actors(soup):
    actor_infos = []
    actor_items = soup.select('ul[class=lst_people] div[class=p_info]')
    for i, item in enumerate(actor_items):
        order = i + 1
        actor_info = parse_actor(item)
        actor_info['order'] = order
        actor_infos.append(actor_info)
    return actor_infos

parse_actors(soup)

[{'name': '라이언 고슬링', 'role': '세바스찬 역', 'order': 1},
 {'name': '엠마 스톤', 'role': '미아 역', 'order': 2},
 {'name': '존 레전드', 'role': '키이스 역', 'order': 3},
 {'name': '로즈마리 드윗', 'role': '로라 역', 'order': 4},
 {'name': 'J.K. 시몬스', 'role': '빌- 재즈클럽 사장 역', 'order': 5},
 {'name': '소노야 미즈노', 'role': '케이틀린 역', 'order': 6},
 {'name': '제시카 로테', 'role': '알렉시스 역', 'order': 7},
 {'name': '칼리 헤르난데스', 'role': '트레이시 역', 'order': 8},
 {'name': '톰 에버렛 스콧', 'role': '데이비드 역', 'order': 9},
 {'name': '핀 위트록', 'role': '그렉 역', 'order': 10},
 {'name': '제이슨 푸치스', 'role': '카를로 역', 'order': 11},
 {'name': '올리비아 해밀턴', 'role': '브리 역', 'order': 12}]

## 수집된 정보를 JSON 형식으로 저장하기

영화 제목과 출연 배우의 정보를 하나의 JSON 에 합치는 함수도 만듭니다.

In [13]:
def scrap_movie_info(url):
    info = {'idx': int(url.split('=')[-1])}
    soup = get_soup(url)

    info['title'] = parse_title(soup)
    info['actors'] = parse_actors(soup)
    return info

scrap_movie_info(url)

{'idx': 134963,
 'title': 'La La Land, 2016',
 'actors': [{'name': '라이언 고슬링', 'role': '세바스찬 역', 'order': 1},
  {'name': '엠마 스톤', 'role': '미아 역', 'order': 2},
  {'name': '존 레전드', 'role': '키이스 역', 'order': 3},
  {'name': '로즈마리 드윗', 'role': '로라 역', 'order': 4},
  {'name': 'J.K. 시몬스', 'role': '빌- 재즈클럽 사장 역', 'order': 5},
  {'name': '소노야 미즈노', 'role': '케이틀린 역', 'order': 6},
  {'name': '제시카 로테', 'role': '알렉시스 역', 'order': 7},
  {'name': '칼리 헤르난데스', 'role': '트레이시 역', 'order': 8},
  {'name': '톰 에버렛 스콧', 'role': '데이비드 역', 'order': 9},
  {'name': '핀 위트록', 'role': '그렉 역', 'order': 10},
  {'name': '제이슨 푸치스', 'role': '카를로 역', 'order': 11},
  {'name': '올리비아 해밀턴', 'role': '브리 역', 'order': 12}]}

이 정보를 JSON 으로 저장합니다. os 를 이용하여 만약 해당 디렉토리가 존재하지 않을 경우 이를 만드는 함수도 미리 만들어 둡니다.

In [14]:
import os

def check_dir(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        print('created {}'.format(dirpath))

라라랜드의 영화 아이디는 `134963` 입니다. dirpath 에 이를 추가하여 JSON 파일의 주소를 만들어 저장합니다. indent 를 0 보다 크게 설정하면 각 정보별로 줄바꿈 및 들여쓰기를 하여 가독성이 좋아집니다. 대신 띄어쓰기에 의해 용량이 조금 커집니다. ensure_ascii=False 로 설정하면 한글이 깨지지 않고 기록됩니다.

In [15]:
import json

dirpath = './scraped_movies/'
idx = 134963
filepath = '{}/{}.json'.format(dirpath, idx)

check_dir(dirpath)
def save_json(obj, path):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

info = scrap_movie_info(url)
save_json(info, filepath)

## 여러 영화에 대하여 정보를 가져오기 (sleep, try-except)

네이버 영화의 url 은 base 에 각 영화의 아이디가 더해지는 형식입니다. 이를 이용하여 원하는 영화들에 대해 제목과 출연 배우를 가져와봅니다.

그런데 영화 별로 정보를 가져오다보면 인터넷 장애 혹은 해당 웹 사이트에서 공격적인 데이터 수집을 막기 위해 접속을 차단할 수도 있습니다. 이를 방지하기 위해서는 각 영화별로 url 을 가져온 뒤, 어느 정도 쉬어주는 것이 좋습니다. 이를 위해 time.sleep 을 이용할 수 있습니다.

In [16]:
for i in range(5):
    print(i)

0
1
2
3
4


time.sleep(t) 은 t 초 동안 작업을 멈추는 것입니다.

In [17]:
import time
for i in range(5):
    print(i)
    time.sleep(0.5)

0
1
2
3
4


try - except 는 예상하지 못한 오류가 발생할 경우에 이를 대처하는 것입니다. 아래 코드는 숫자가 아닌 글자를 정수로 casting 하기 때문에 오류가 발생하여 중간에 멈춥니다.

In [18]:
for i in range(5):    
    s = str(i) if i != 3 else 'a%d' % i
    j = int(s)
    print('s = %s, j = %d' % (s, j))

s = 0, j = 0
s = 1, j = 1
s = 2, j = 2


ValueError: invalid literal for int() with base 10: 'a3'

그러나 try - except 를 이용하면 해당 부분에 대해 어떤 예외가 발생하였는지 메시지를 출력하고 다음 loop 으로 넘어갑니다.

In [19]:
for i in range(5):
    try:
        s = str(i) if i != 3 else 'a%d' % i
        j = int(s)
        print('s = %s, j = %d' % (s, j))
    except Exception as e:
        print(e)
        continue

s = 0, j = 0
s = 1, j = 1
s = 2, j = 2
invalid literal for int() with base 10: 'a3'
s = 4, j = 4


아래의 세 영화에 대하여 정보를 가져옵니다.

In [20]:
idxs = [134963, 24830, 130903]

url_base = 'https://movie.naver.com/movie/bi/mi/detail.nhn?code={}'
dirpath = './scraped_movies/'

check_dir(dirpath)

for idx in idxs:
    url = url_base.format(idx)
    info = scrap_movie_info(url)
    filepath = '{}/{}.json'.format(dirpath, idx)
    save_json(info, filepath)
    print('Scraped from {}'.format(url))
    time.sleep(1)

Scraped from https://movie.naver.com/movie/bi/mi/detail.nhn?code=134963
Scraped from https://movie.naver.com/movie/bi/mi/detail.nhn?code=24830
Scraped from https://movie.naver.com/movie/bi/mi/detail.nhn?code=130903


## 영화평 가져오기

영화평 페이지에 마우스를 올린 다음에, 링크 복사를 하면 아래와 같은 url 을 얻을 수 있습니다. 우리는 이 페이지에서도 댓글을 파싱하는 함수를 만들어 봅니다.

In [21]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=134963&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2'
url

'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=134963&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2'

In [22]:
def parse_comments_from_a_page(url):
    soup = get_soup(url)
    comment_items = soup.select('div[class=score_result] li')
    comments = [parse_comment_item(item) for item in comment_items]
    return comments

def parse_comment_item(item):
    return {
        'score': int(item.select('div[class=star_score]')[0].text),
        'text': normalize(item.select('div[class=score_reple] p')[0].text)
    }

comments = parse_comments_from_a_page(url)

In [23]:
comments

[{'score': 10,
  'text': '이 영화의 백미는 마지막 부분에 남녀 주인공들의 IF 장면들... 이렇게 되지 못한 결말을 맞지만 그래도 서로의 꿈을 이루고 인정하는 듯한 둘만의 미소,,,, 뭔가 가슴 찡했다...'},
 {'score': 10,
  'text': '위플래시가 음악에 대한 광기라면 이 영화는 열정이다. 같은날 개봉되는 삼류 국산영화는 범접조차 할수 없다'},
 {'score': 7, 'text': '관람객 슬프기 짝이 없는 상상.'},
 {'score': 9,
  'text': '관람객 아이러니컬하게도, 꿈이라는 건 이루고 난 뒤보다 이루기 전까지가 실은 가장 행복했다는 걸, 이루고 난 뒤에야 깨닫는다는 인생의 진실을 가슴 아프게 그리고 아름답게 보여주는 영화'},
 {'score': 10, 'text': '관람객 서로의 꿈을 돌보아주고 키워가며 각자의 길을 응원하는 최고의 뮤지컬.'},
 {'score': 8, 'text': '관람객 다시 돌아간다면 지금과 달랐을까?'},
 {'score': 10, 'text': '관람객 왤케눈물이나오는지..ㅠㅠ'},
 {'score': 10,
  'text': '다미엔 차젤레 감독은 관객에게 황홀한 마법을 걸었다. 재즈에 대한 애정을 담아 연출했고 꿈 꾸는 모든 이들을 위한 영화. 롱테이크 오프닝 시퀀스, OST부터 이미 매료되었고 엔딩까지 한 편의 동화 같은 현실 이야기....'},
 {'score': 10, 'text': '관람객 영화 시작부터 깜짝놀랐다. 지금까지 본 음악영화 중에서 최고다.'},
 {'score': 10,
  'text': '시간이 지나도 우리가 함께 꿈꾸었다는 사실은 변하지 않을 것이다. 그것이 비록 서로 다른 꿈이었을지라도...'}]

영화평 url 을 살펴보면 구조가 있습니다.  page 는 페이지 번호이며, type=after 는 관람후 평점을 의미합니다. 이 정보를 바탕으로 url_base 를 만든 뒤, 페이지를 증가시키며 댓글을 수집하는 함수를 만듭니다. 여러 번 url get 을 할 경우에는 반드시 sleep 을 하는 것이 좋습니다. 그렇지 않으면 공격성 접근으로 차단 당할 수 있습니다.

In [24]:
comment_url_base = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'

def parse_comments(idx, max_page=10, sleep=1.0, verbose=True):
    comments = []
    for page in range(1, max_page + 1):
        url = comment_url_base.format(idx, page)
        comments += parse_comments_from_a_page(url)
        if verbose:
            print('scraped comments from id={}, page={} / {}'.format(idx, page, max_page))
        time.sleep(sleep)
    return comments

comments = parse_comments(134963)

scraped comments from id=134963, page=1 / 10
scraped comments from id=134963, page=2 / 10
scraped comments from id=134963, page=3 / 10
scraped comments from id=134963, page=4 / 10
scraped comments from id=134963, page=5 / 10
scraped comments from id=134963, page=6 / 10
scraped comments from id=134963, page=7 / 10
scraped comments from id=134963, page=8 / 10
scraped comments from id=134963, page=9 / 10
scraped comments from id=134963, page=10 / 10


In [25]:
len(comments)

100

In [26]:
comments[:3]

[{'score': 9, 'text': "'그래도 우리 잘 해냈지?'라고 말하는 듯한 마지막 눈빛교환이 잊혀지질 않는다"},
 {'score': 9,
  'text': '와..그냥 감탄밖엔 안 나온다. 영상미며 음악이며..조폭, 건달, 검사, 신파 좋아하고 실험을 꺼리는 한국의 기성 영화인들 반성해라. 이제 겨우 두 편 만든 신인감독이 이 정도다'},
 {'score': 7, 'text': '처음 들을때는 몰랐다 이 음악이 슬퍼질거란걸'}]

## headers (user-agent)

그런데 어떤 사이트들은 웹브라우저가 아닌 접근을 허용하지 않는 경우들도 있습니다. IMDB 사이트가 그러합니다. 이때는 requests 를 보낼 때, 나는 공격성 접근이 아니라는 정보를 함께 전달해야 합니다. 이 정보는 headers 에 입력합니다. 그래서 앞서 get_soup 함수가 아래처럼 구현되었던 것입니다.

```python
requests.get(url, headers=headers)
```

headers 를 이용할 때에는 아래처럼 get_soup 함수에 headers 를 입력합니다.

In [27]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
soup = get_soup(url, headers)